In [8]:
import scipy.sparse as spr
import scipy.sparse.linalg as spla
import sys
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.real   as r
import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

import pyoti.static.onumm1n1    as dual
import pyoti.static.mdnum2      as md2
import pyoti.static.mdnum3      as md3
import pyoti.static.mdnum5      as md5
import pyoti.static.onumm1n10   as om1n10
import pyoti.static.onumm1n2    as om1n2
import pyoti.static.onumm1n5    as om1n5
import pyoti.static.onumm5n5    as om5n5
import pyoti.static.onumm2n2    as om2n2
import pyoti.static.onumm2n5    as om2n5
import pyoti.static.onumm3n3    as om3n3




%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np

import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')

global times
from timeit import default_timer as time

In [2]:
def cylinder(ri, ro, he = 1.0, element_order = 1, quads = False, 
           quad_incomplete = 1, quad_linear = 1, structured = False, save=False):
    """
    PORPUSE: Define a cylinder section mesh.
    """
    #***************************************************************************************************
    import gmsh

    gmsh.initialize()
    # gmsh.fltk.initialize()

    # Lets create a simple square element:
    model = gmsh.model
    geo   = model.geo
    option= gmsh.option

    P1 = geo.addPoint( 0.0, 0.0, 0.0, he, 1)
    P2 = geo.addPoint( 0.0,  ri, 0.0, he, 2)
    P3 = geo.addPoint(  ri, 0.0, 0.0, he, 3)
    P4 = geo.addPoint( 0.0,  ro, 0.0, he, 4)
    P5 = geo.addPoint(  ro, 0.0, 0.0, he, 5)

    C1 = geo.addCircleArc(P2,P1,P3)
    C2 = geo.addCircleArc(P5,P1,P4)
    
    geo.remove([P1])
    
    L1 = geo.addLine(P4,P2) # Left
    L2 = geo.addLine(P3,P5) # bottom

    loop1 = geo.addCurveLoop([C1,L2,C2,L1])

    surface = geo.addPlaneSurface([loop1])

    model.addPhysicalGroup( 0, [P1],        100 )
    model.addPhysicalGroup( 0, [P2],        101 )
    model.addPhysicalGroup( 0, [P3],        102 )
    model.addPhysicalGroup( 0, [P4],        103 )
    model.addPhysicalGroup( 0, [P5],        104 )

    model.addPhysicalGroup( 1, [C1],        201 )
    model.addPhysicalGroup( 1, [C2],        202 )
    model.addPhysicalGroup( 1, [L1],        203 )
    model.addPhysicalGroup( 1, [L2],        204 )

    model.addPhysicalGroup( 2, [surface],   301 )
    
    model.setPhysicalName( 0, 100, "center"  )
    model.setPhysicalName( 0, 101, "ri_left"  )
    model.setPhysicalName( 0, 102, "ri_right" )
    model.setPhysicalName( 0, 103, "ro_left"  )
    model.setPhysicalName( 0, 104, "ro_right" )

    model.setPhysicalName( 1, 201, "ri"    )
    model.setPhysicalName( 1, 202, "ro"    )
    model.setPhysicalName( 1, 203, "left"  )
    model.setPhysicalName( 1, 204, "right" )

    model.setPhysicalName( 2, 301, "domain"      )

    
    geo.synchronize()

    option.setNumber('Mesh.ElementOrder',element_order)


    # Recombine if quads are wanted.
    if quads:

        # Set body to recombine into quads.
        option.setNumber('Mesh.SecondOrderIncomplete',quad_incomplete)
        option.setNumber('Mesh.SecondOrderLinear',    quad_linear    )
        option.setNumber('Mesh.RecombineAll',         1)

    else:

        option.setNumber('Mesh.SecondOrderIncomplete',quad_incomplete)
        option.setNumber('Mesh.SecondOrderLinear',    quad_linear    )
        option.setNumber('Mesh.RecombineAll',         0)

    # end if 

    if structured:
        nels_L = int((ro-ri)/he)+1
        nels_C = int((ri*np.pi*2)/(4*he))+1
        model.mesh.setTransfiniteCurve(L1,nels_L)
        model.mesh.setTransfiniteCurve(L2,nels_L)
        model.mesh.setTransfiniteCurve(C1,nels_C)
        model.mesh.setTransfiniteCurve(C2,nels_C)
        model.mesh.setTransfiniteSurface(surface)#,"left",[P2,P3,P4])
    # end if 

    model.mesh.generate(2)

    if save:
        gmsh.write("cylinder.msh")
    # end if 

    Th = fem.mesh.from_gmsh(gmsh)
#     Th = None

    gmsh.finalize()

    return Th

#-----------------------------------------------------------------------------------------------------

In [3]:
#*****************************************************************************************************
def solve_2d_linear_elasticity(Th, E, nu, ri, Pi, ro, Po, stats=True):
    global times
    from timeit import default_timer as time
    
    start_time = time()
    
    mu = E/(2*(1+nu))
    
    c1 = E*nu/((1+nu)*(1-2*nu))
    c2 = 2*mu
    c3 = mu
    c4 = c1+c2
    
    ndim_analysis = 2
    els = Th.elements[2]

    fem.end_elements()
    
    nNodes = Th.x.shape[0]
    nDOF = 2*nNodes
    
    K = alg.lil_matrix((nDOF,nDOF))
    f = alg.zeros((nDOF,1))
    
    K[nNodes,nNodes] = 1.0 # Removes "center" node.
    K[0,0] = 1.0 # Removes "center" node.
    
    # 
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = True)

            fh = alg.zeros( ( elem.nbasis, 1 ) )
            
            # Temps
            
            NxT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
            NyT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
            
            NxNx = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NxNy = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NyNx = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NyNy = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )

            NN_tmp1 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NN_tmp2 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NN_tmp3 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            
            tmp11 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp12 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp21 = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            tmp22 = alg.zeros( ( elem.nbasis, elem.nbasis ) )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()

            Nx = elem.Nx
            Ny = elem.Ny
            N  = elem.N
            
            alg.transpose(Nx, out = NxT)
            alg.transpose(Ny, out = NyT)
            
            alg.dot( NxT, Nx, out = NxNx )
            alg.dot( NyT, Ny, out = NyNy )
            alg.dot( NyT, Nx, out = NyNx )
            alg.dot( NxT, Ny, out = NxNy )

            # problem = intV( c1*dx(vx)*dx(ux) + c1*dy(vy)*dx(ux) ) + \
            #           intV( c1*dx(vx)*dy(uy) + c1*dy(vy)*dy(uy) ) + \
            #           intV( c2*dx(vx)*dx(ux) + c2*dy(vy)*dy(uy) ) + \
            #           intV( c3*dy(vx)*dy(ux) + c3*dx(vy)*dy(ux) ) + \
            #           intV( c3*dy(vx)*dx(uy) + c3*dx(vy)*dx(uy) ) + \
            #           intS( 'in' , fxi*vx + fyi*vy ) + \
            #           intS( 'out', fxo*vx + fyo*vy ) + \
            #           on(   'left' , ux, 0.0) + \
            #           on(   'right', uy, 0.0)
            #  
            # c1*NxNx + c2*NxNx + c3*NyNy # vx, ux
            # c1*NxNy + c3*NyNx           # vx, uy
            # c1*NyNx + c3*NxNy           # vy, ux
            # c1*NyNy + c2*NyNy + c3*NxNx # vy, uy
            
            # c4 = c1+c2

#             tmp11.set(0)
#             tmp12.set(0)
#             tmp21.set(0)
#             tmp22.set(0)
            
            # alg.gauss_integrate( c4*NxNx + c3*NyNy , elem.dV,out=tmp11) # vx,ux            
            alg.mul(      c4,    NxNx, out=NN_tmp1 )
            alg.mul(      c3,    NyNy, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp11 ) # vx,ux
            
            # alg.gauss_integrate( c1*NxNy + c3*NyNx , elem.dV,out=tmp12) # vx,uy
            alg.mul(      c1,    NxNy, out=NN_tmp1 )
            alg.mul(      c3,    NyNx, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp12 ) # vx,uy
            
            # alg.gauss_integrate( c1*NyNx + c3*NxNy , elem.dV,out=tmp21) # vy,ux
            alg.mul(      c1,    NyNx, out=NN_tmp1 )
            alg.mul(      c3,    NxNy, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp21 ) # vy,ux
            
            # alg.gauss_integrate( c4*NyNy + c3*NxNx , elem.dV,out=tmp22) # vy,uy
            alg.mul(      c4,    NyNy, out=NN_tmp1 )
            alg.mul(      c3,    NxNx, out=NN_tmp2 )
            alg.sum( NN_tmp1, NN_tmp2, out=NN_tmp3 )
            alg.gauss_integrate( NN_tmp3, elem.dV, out = tmp22 ) # vy,uy
            

                      
            # assemble globals
            
            for k in range(elems.size):
        
                ii=int(elems[k])
                
                i1 = ii
                i2 = ii + nNodes # Move DOF for second variable
                
                for l in range(elems.size):

                    jj=int(elems[l])
                    
                    j1 = jj
                    j2 = jj + nNodes # Move DOF for second variable
                    
                    K[i1,j1] = K[i1,j1] + tmp11[k,l]
                    
                    K[i1,j2] = K[i1,j2] + tmp12[k,l]
                    
                    K[i2,j1] = K[i2,j1] + tmp21[k,l]
                    
                    K[i2,j2] = K[i2,j2] + tmp22[k,l]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()   

    end_assmbly_time = time()
    
    
    # 
    els = Th.group_names['ri']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            nxel = alg.zeros( ( elem.nbasis, 1 ) )
            xel  = alg.zeros( ( elem.nbasis, 1 ) )
            nyel = alg.zeros( ( elem.nbasis, 1 ) )
            yel  = alg.zeros( ( elem.nbasis, 1 ) )
            

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)  
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ri
            nyel = yel/ri
            
            
            tmp1 = alg.gauss_integrate( (Pi)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (Pi)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            
#             print("tmp1[0,0].order",tmp1[0,0].order)
#             print("tmp2[0,0].order",tmp2[0,0].order)
            
            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for
#     print("f.order", f.order)

    fem.end_elements()  
    
    els = Th.group_names['ro']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            fh = alg.zeros( ( elem.nbasis, 1 ) )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ro
            nyel = yel/ro
            
            
            tmp1 = alg.gauss_integrate( (-Po)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (-Po)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            

            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for
#     print("f.order", f.order)

    fem.end_elements()
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30
    
    # Every 1D node has dirichlet bc = 0
    els = Th.group_names['right']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_) # uy
            i2 = int(ii_)+nNodes # uy

            K[i2,i2] = TGV
            f[i2,0] = 0.0
            
#             K[i1,i1] = TGV
#             f[i1,0] = 0.1*TGV
        
        # end for 
        
        
    # end for
    
    els = Th.group_names['left']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            ii = int(ii_) # ux
            K[ii,ii] = TGV
            # end for
            f[ii,0] = 0.0
        # end for 
        
    # end for
    end_bc_time = time()
    
#     print("f.order", f.order)
    
    u = alg.solve(K.tocsr(),f)
    # u = alg.solve_sparse_tests(K.tocsr(),f)
    
    end_solve_time = time()
    
    if stats:
        times['assembly'].append(  end_assmbly_time - start_time       )
        times['bc'].append( end_bc_time      - end_assmbly_time  )
        times['solve'].append(  end_solve_time   - end_bc_time      )
        times['total'].append( end_solve_time   - start_time        )
        
#         print("Assembly time:  {0:.6f} s ".format( end_assmbly_time - start_time       ) )
#         print("Boundary time:  {0:.6f} s ".format( end_bc_time      - end_assmbly_time ) )
#         print("Solution time:  {0:.6f} s ".format( end_solve_time   - end_bc_time      ) )
        print("Total run time: {0:.6f} s ".format( end_solve_time   - start_time       ) )
        print()
    # end if 

    return u,K,f

    #-----------------------------------------------------------------------------------------------------

In [4]:
#*****************************************************************************************************
def solve_2d_linear_elasticity_bruteforce(Th, E, nu, ri, Pi, ro, Po, stats=True):
    global times
    from timeit import default_timer as time
    
    start_time = time()
    
    mu = E/(2*(1+nu))
    
    c1 = E*nu/((1+nu)*(1-2*nu))
    c2 = 2*mu
    c3 = mu
    c4 = c1 + c2
    
    ndim_analysis = 2
    els = Th.elements[2]

    fem.end_elements()
    
    nNodes = Th.x.shape[0]
    nDOF = 2*nNodes
    
    K = alg.lil_matrix((nDOF,nDOF))
    f = alg.zeros((nDOF,1))
    
    K[nNodes,nNodes] = 1.0 # Removes "center" node.
    K[0,0] = 1.0 # Removes "center" node.
    
    # 
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = True)

            fh = alg.zeros( ( elem.nbasis, 1 ) )
            
            # Temps
            
#             NxT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
#             NyT = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()

            Nx = elem.Nx
            Ny = elem.Ny
            N  = elem.N
            
            NxNx = alg.dot(alg.transpose(Nx),Nx)
            NyNy = alg.dot(alg.transpose(Ny),Ny)
            NyNx = alg.dot(alg.transpose(Ny),Nx)
            NxNy = alg.dot(alg.transpose(Nx),Ny)

#             problem = intV( c1*dx(vx)*dx(ux) + c1*dy(vy)*dx(ux) ) + \
#                       intV( c1*dx(vx)*dy(uy) + c1*dy(vy)*dy(uy) ) + \
#                       intV( c2*dx(vx)*dx(ux) + c2*dy(vy)*dy(uy) ) + \
#                       intV( c3*dy(vx)*dy(ux) + c3*dx(vy)*dy(ux) ) + \
#                       intV( c3*dy(vx)*dx(uy) + c3*dx(vy)*dx(uy) ) + \
#                       intS( 'in' , fxi*vx + fyi*vy ) + \
#                       intS( 'out', fxo*vx + fyo*vy ) + \
#                       on(   'left' , ux, 0.0) + \
#                       on(   'right', uy, 0.0)

#             c1*NxNx + c2*NxNx + c3*NyNy # vx, ux
#             c1*NxNy + c3*NyNx           # vx, uy
#             c1*NyNx + c3*NxNy           # vy, ux
#             c1*NyNy + c2*NyNy + c3*NxNx # vy, uy
            
            tmp11 = alg.gauss_integrate( c4*NxNx + c3*NyNy , elem.dV) # vx,ux
            tmp12 = alg.gauss_integrate( c1*NxNy + c3*NyNx , elem.dV) # vx,uy
            tmp21 = alg.gauss_integrate( c1*NyNx + c3*NxNy , elem.dV) # vy,ux
            tmp22 = alg.gauss_integrate( c4*NyNy + c3*NxNx , elem.dV) # vy,uy
            
                      
            # assemble globals
            
            for k in range(elems.size):
        
                ii=int(elems[k])
                
                i1 = ii
                i2 = ii + nNodes # Move DOF for second variable
                
                for l in range(elems.size):

                    jj=int(elems[l])
                    
                    j1 = jj
                    j2 = jj + nNodes # Move DOF for second variable
                    
                    K[i1,j1] = K[i1,j1] + tmp11[k,l]
                    
                    K[i1,j2] = K[i1,j2] + tmp12[k,l]
                    
                    K[i2,j1] = K[i2,j1] + tmp21[k,l]
                    
                    K[i2,j2] = K[i2,j2] + tmp22[k,l]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()   
    
    
    
    end_assmbly_time = time()
    
    
    # 
    els = Th.group_names['ri']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            nxel = alg.zeros( ( elem.nbasis, 1 ) )
            xel  = alg.zeros( ( elem.nbasis, 1 ) )
            nyel = alg.zeros( ( elem.nbasis, 1 ) )
            yel  = alg.zeros( ( elem.nbasis, 1 ) )
            

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)  
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ri
            nyel = yel/ri
            
            
            tmp1 = alg.gauss_integrate( (Pi)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (Pi)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            
            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()  
    
    els = Th.group_names['ro']['members'][0]
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = False)

            fh = alg.zeros( ( elem.nbasis, 1 ) )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elems)
            elem.compute_jacobian()
            
            N  = alg.transpose(elem.N)
            
            for k in range(elems.size):
                xel[k,0] = Th.x[int(elems[k]),0]
                yel[k,0] = Th.y[int(elems[k]),0]
            # end for 
            
            nxel = xel/ro
            nyel = yel/ro
            
            
            tmp1 = alg.gauss_integrate( (-Po)*alg.dot_product(nxel,N)*N, elem.dV) # vx,ux
            tmp2 = alg.gauss_integrate( (-Po)*alg.dot_product(nyel,N)*N, elem.dV) # vy,ux
            

            # assemble globals            
            for k in range(elems.size):
        
                ii=int(elems[k])

                i1 = ii
                i2 = ii + nNodes
                
                f[i1,0] = f[i1,0] + tmp1[k,0]
                f[i2,0] = f[i2,0] + tmp2[k,0]

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30
    
    # Every 1D node has dirichlet bc = 0
    els = Th.group_names['right']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_) # uy
            i2 = int(ii_)+nNodes # uy

            K[i2,i2] = TGV
            f[i2,0] = 0.0
            
#             K[i1,i1] = TGV
#             f[i1,0] = 0.1*TGV
        
        # end for 
        
        
    # end for
    
    els = Th.group_names['left']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            ii = int(ii_) # ux
            K[ii,ii] = TGV
            # end for
            f[ii,0] = 0.0
        # end for 
        
    # end for
    end_bc_time = time()
    
   
    u = alg.solve(K.tocsr(),f)
#     u = alg.solve_sparse_tests(K.tocsr(),f)
    
    end_solve_time = time()
    
    if stats:
        times['assembly'].append(  end_assmbly_time - start_time       )
        times['bc'].append( end_bc_time      - end_assmbly_time  )
        times['solve'].append(  end_solve_time   - end_bc_time      )
        times['total'].append( end_solve_time   - start_time        )
        
#         print("Assembly time:  {0:.6f} s ".format( end_assmbly_time - start_time       ) )
#         print("Boundary time:  {0:.6f} s ".format( end_bc_time      - end_assmbly_time ) )
#         print("Solution time:  {0:.6f} s ".format( end_solve_time   - end_bc_time      ) )
        print("Total run time: {0:.6f} s ".format( end_solve_time   - start_time       ) )
        print()
    # end if 

    return u,K,f

    #-----------------------------------------------------------------------------------------------------
    
def assemble_globals(Kel,fel,Kgl,fgl,elem):
    
    
    for i in range(elem.size):
        
        ii=int(elem[i])
        
        for j in range(elem.size):

            jj=int(elem[j])
        
            Kgl[ii,jj] = Kgl[ii,jj] + Kel[i,j]
            
        # end for 
        fgl[ii,0] = fgl[ii,0] + fel[i,0]
        
    # end for 

    
def analytic_solution(Th,E,nu,ri,pi,ro,po):
    
    A = ri**2*ro**2*(po-pi)/(ro**2-ri**2)
    B = (ri**2*pi - ro**2*po)/(ro**2-ri**2)
    
    x = Th.x
    y = Th.y
    
    r =  (x**2+y**2)**0.5
    
    ur = (1+nu)*r/E*((1-2*nu)*B-A/(r**2))
    
    ux = ur*x/r
    uy = ur*y/r
    
    return ux, uy

In [5]:
# r, dual, md2, md3, md5, md10, om1n10, 
# om1n2, om1n5, om5n5, om2n5, om2n2, om3n3, oti
fem.set_global_algebra(r)
alg = fem.get_global_algebra()





start_time = time()


# Th = cylinder(1,2,he=0.004,quads=False,save=True, structured=True)
Th = cylinder(1,2,he=0.0027,quads=False,save=True, structured=True)
# Th = cylinder(1,2,he=0.01,quads=False,save=True, structured=True)



end_time = time()

print(end_time - start_time,"seg")



Th

/Users/maristi7/opt/anaconda3/envs/pyoti/lib/python3.7/site-packages/numpy/ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


1.802298133999999 seg


< mesh (pyoti.real) object with 215923 nodes, 431847 elements of types ( point1 (5), line2 (1902), tri3 (429940) ) >

In [16]:
md5.number(3)

3.0000 + 0.0000 * e([1]) + 0.0000 * e([2]) + 0.0000 * e([3]) + 0.0000 * e([4]) + 0.0000 * e([5]) + 0.0000 * e([1,2]) + 0.0000 * e([1,3]) + 0.0000 * e([2,3]) + 0.0000 * e([1,4]) + 0.0000 * e([2,4]) + 0.0000 * e([3,4]) + 0.0000 * e([1,5]) + 0.0000 * e([2,5]) + 0.0000 * e([3,5]) + 0.0000 * e([4,5]) + 0.0000 * e([1,2,3]) + 0.0000 * e([1,2,4]) + 0.0000 * e([1,3,4]) + 0.0000 * e([2,3,4]) + 0.0000 * e([1,2,5]) + 0.0000 * e([1,3,5]) + 0.0000 * e([2,3,5]) + 0.0000 * e([1,4,5]) + 0.0000 * e([2,4,5]) + 0.0000 * e([3,4,5]) + 0.0000 * e([1,2,3,4]) + 0.0000 * e([1,2,3,5]) + 0.0000 * e([1,2,4,5]) + 0.0000 * e([1,3,4,5]) + 0.0000 * e([2,3,4,5]) + 0.0000 * e([1,2,3,4,5])

In [18]:
om2n5.number(3)

3.0000 + 0.0000 * e([1]) + 0.0000 * e([2]) + 0.0000 * e([[1,2]]) + 0.0000 * e([1,2]) + 0.0000 * e([[2,2]]) + 0.0000 * e([[1,3]]) + 0.0000 * e([[1,2],2]) + 0.0000 * e([1,[2,2]]) + 0.0000 * e([[2,3]]) + 0.0000 * e([[1,4]]) + 0.0000 * e([[1,3],2]) + 0.0000 * e([[1,2],[2,2]]) + 0.0000 * e([1,[2,3]]) + 0.0000 * e([[2,4]]) + 0.0000 * e([[1,5]]) + 0.0000 * e([[1,4],2]) + 0.0000 * e([[1,3],[2,2]]) + 0.0000 * e([[1,2],[2,3]]) + 0.0000 * e([1,[2,4]]) + 0.0000 * e([[2,5]])

In [6]:
times = {}
times['assembly'] = []
times['bc'] = []
times['solve'] = []
times['total'] = []

E  = alg.number(21e5)#+alg.e(1,order=10)#,order=1)#,1,10,2,1000
nu = alg.number(0.28)#+alg.e(2,order=5)
ri = alg.number(1)
Pi = alg.number(10) #+alg.e(1,order=10)# Pi = alg.number(1400)
ro = alg.number(2)
Po = alg.number(1000)#+alg.e(4,order=5)

for i in range(1):
#     u_res,K,f = solve_2d_linear_elasticity(Th,E,nu,ri,Pi,ro,Po)
    u_res,K,f = solve_2d_linear_elasticity(Th,E,nu,ri,Pi,ro,Po)
#end for

print('\n\nTimes:'+str(type(u_res)))
print("- Avg Assembly time:  {0:.6f} s ".format(np.average(times['assembly'] ) ) )
print("- Avg Boundary time:  {0:.6f} s ".format(np.average(times['bc'])      ) )
print("- Avg Solution time:  {0:.6f} s ".format(np.average(times['solve'])   ) )

print("- Avg Total run time: {0:.6f} s ".format(np.average(times['total'])   ) )

Total run time: 63.176529 s 



Times:<class 'pyoti.real.dmat'>
- Avg Assembly time:  18.822710 s 
- Avg Boundary time:  0.019151 s 
- Avg Solution time:  44.334668 s 
- Avg Total run time: 63.176529 s 


In [9]:
K

<(431846, 431846) sparse matrix of OTI numbers with 
         6030578 stored elements in LInked List format>

In [11]:
start_t = time()

Kcsc = K.tocsr().real.tocsc()
Kcsc

lu = spla.splu(Kcsc)
end_t = time()

print("Time to factorize: {0:.4f}s".format(end_t-start_t))



Time to factorize: 50.8169s


In [13]:
%timeit lu.solve(f.real)

408 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
# XPS  (429940) Triangles

# Times:<class 'pyoti.real.dmat'>
# - Avg Assembly time:  11.113099 s 
# - Avg Boundary time:  0.009709 s 
# - Avg Solution time:  17.195912 s 
# - Avg Total run time: 28.318721 s 

# Times:<class 'pyoti.static.onumm1n1.omatm1n1'> (Duals)
# - Avg Assembly time:  17.557875 s 
# - Avg Boundary time:  0.018980 s 
# - Avg Solution time:  18.858839 s 
# - Avg Total run time: 36.435693 s 

# Times:<class 'pyoti.static.mdnum2.mdmat2'>
# - Avg Assembly time:  20.367134 s 
# - Avg Boundary time:  0.030285 s 
# - Avg Solution time:  20.444894 s 
# - Avg Total run time: 40.842314 s 

# Times:<class 'pyoti.static.mdnum3.mdmat3'>
# - Avg Assembly time:  21.308258 s 
# - Avg Boundary time:  0.022566 s 
# - Avg Solution time:  23.918862 s 
# - Avg Total run time: 45.249685 s

# Times:<class 'pyoti.static.mdnum5.mdmat5'>
# - Avg Assembly time:  47.205461 s 
# - Avg Boundary time:  0.046123 s 
# - Avg Solution time:  36.910316 s 
# - Avg Total run time: 84.161900 s 







# Times:<class 'pyoti.static.onumm1n5.omatm1n5'>
# - Avg Assembly time:  21.202655 s 
# - Avg Boundary time:  0.030977 s 
# - Avg Solution time:  26.509539 s 
# - Avg Total run time: 47.743171 s 

# Times:<class 'pyoti.static.onumm2n5.omatm2n5'>
# - Avg Assembly time:  31.470576 s 
# - Avg Boundary time:  0.028044 s 
# - Avg Solution time:  31.799710 s 
# - Avg Total run time: 63.298330 s 

# Times:<class 'pyoti.static.onumm1n10.omatm1n10'> (10 derivs)
# - Avg Assembly time:  26.752567 s 
# - Avg Boundary time:  0.029071 s 
# - Avg Solution time:  48.612258 s 
# - Avg Total run time: 75.393896 s 





# Times:<class 'pyoti.sparse.matso'>  (No derivs)
# - Avg Assembly time:  35.950992 s 
# - Avg Boundary time:  0.023023 s 
# - Avg Solution time:  18.104725 s 
# - Avg Total run time: 54.078741 s 

# Times:<class 'pyoti.sparse.matso'> (4 var order 1) (4 derivs)
# - Avg Assembly time:  43.191356 s 
# - Avg Boundary time:  0.024698 s 
# - Avg Solution time:  22.919118 s 
# - Avg Total run time: 66.135172 s 

# Times:<class 'pyoti.sparse.matso'> (4 var order 2) (14 derivs)
# - Avg Assembly time:  48.721740 s 
# - Avg Boundary time:  0.025488 s 
# - Avg Solution time:  32.897843 s 
# - Avg Total run time: 81.645071 s 

# Times:<class 'pyoti.sparse.matso'> (4 var order 3) (34 derivs)
# - Avg Assembly time:  53.962597 s 
# - Avg Boundary time:  0.027144 s 
# - Avg Solution time:  51.565640 s 
# - Avg Total run time: 105.555382 s

# Times:<class 'pyoti.sparse.matso'> (4 var order 4) (69 derivs)
# - Avg Assembly time:  60.701763 s 
# - Avg Boundary time:  0.027271 s 
# - Avg Solution time:  79.216957 s 
# - Avg Total run time: 139.945991 s 


# Times:<class 'pyoti.sparse.matso'> (4 var order 5)  (126 derivs)
# - Avg Assembly time:  65.937784 s 
# - Avg Boundary time:  0.028036 s 
# - Avg Solution time:  120.965043 s 
# - Avg Total run time: 186.930863 s 

# Times:<class 'pyoti.sparse.matso'> (1 var order 10) (10 derivs - E)
# - Avg Assembly time:  74.959733 s 
# - Avg Boundary time:  0.023045 s 
# - Avg Solution time:  459.551980 s 
# - Avg Total run time: 534.534759 s 

# Times:<class 'pyoti.sparse.matso'> (1 var order 10) (10 derivs - Pi)
# - Avg Assembly time:  36.388555 s 
# - Avg Boundary time:  0.026632 s 
# - Avg Solution time:  17.867372 s 
# - Avg Total run time: 54.282560 s 

In [52]:
f.order

0

In [48]:
h = coti.get_dHelp()
h.get_ndir_total(4,5)

126

In [ ]:
# (429940) Triangles

# Times:<class 'pyoti.real.dmat'>
# - Avg Assembly time:  14.577711 s 
# - Avg Boundary time:  0.014906 s 
# - Avg Solution time:  27.854472 s 
# - Avg Total run time: 42.447088 s 

# Times:<class 'pyoti.static.onumm1n1.omatm1n1'> (DUAL)
# - Avg Assembly time:  21.757590 s 
# - Avg Boundary time:  0.021905 s 
# - Avg Solution time:  28.765078 s 
# - Avg Total run time: 50.544573 s 

# Times:<class 'pyoti.static.mdnum2.mdmat2'>
# - Avg Assembly time:  23.798955 s 
# - Avg Boundary time:  0.023322 s 
# - Avg Solution time:  31.539603 s 
# - Avg Total run time: 55.361880 s 

# Times:<class 'pyoti.static.mdnum3.mdmat3'>
# - Avg Assembly time:  25.712955 s 
# - Avg Boundary time:  0.023846 s 
# - Avg Solution time:  35.534308 s 
# - Avg Total run time: 61.271109 s 

# Times:<class 'pyoti.static.mdnum5.mdmat5'>
# - Avg Assembly time:  51.487040 s 
# - Avg Boundary time:  0.034568 s 
# - Avg Solution time:  57.487883 s 
# - Avg Total run time: 109.009490 s 





    
# Times:<class 'pyoti.static.onumm1n2.omatm1n2'>
# - Avg Assembly time:  23.040064 s 
# - Avg Boundary time:  0.026647 s 
# - Avg Solution time:  30.011716 s 
# - Avg Total run time: 53.078426 s 

# Times:<class 'pyoti.static.onumm2n2.omatm2n2'>
# - Avg Assembly time:  24.428853 s 
# - Avg Boundary time:  0.025241 s 
# - Avg Solution time:  32.735985 s 
# - Avg Total run time: 57.190079 s 

# Times:<class 'pyoti.static.onumm3n3.omatm3n3'>
# - Avg Assembly time:  32.978307 s 
# - Avg Boundary time:  0.029697 s 
# - Avg Solution time:  42.746830 s 
# - Avg Total run time: 75.754835 s 

# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  34.033384 s 
# - Avg Boundary time:  0.032038 s 
# - Avg Solution time:  70.974737 s 
# - Avg Total run time: 105.040159 s 

In [78]:
# Real solve time:
# Times:<class 'pyoti.real.dmat'>
# - Avg Assembly time:  26.811997 s 
# - Avg Boundary time:  0.018043 s 
# - Avg Solution time:  38.632964 s 
# - Avg Total run time: 65.463003 s 

# Std:
# Times:<class 'pyoti.static.onumm1n1.omatm1n1'>
# - Avg Assembly time:  35.117580 s 
# - Avg Boundary time:  0.027515 s 
# - Avg Solution time:  43.530336 s 
# - Avg Total run time: 78.675431 s 

# Times:<class 'pyoti.static.mdnum2.mdmat2'>
# - Avg Assembly time:  38.789590 s 
# - Avg Boundary time:  0.112258 s 
# - Avg Solution time:  49.898920 s 
# - Avg Total run time: 88.800768 s 



# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  56.547210 s 
# - Avg Boundary time:  0.147916 s 
# - Avg Solution time:  137.795723 s 
# - Avg Total run time: 194.490850 s 

# Times:<class 'pyoti.sparse.matso'> Compute derivatives of order 10.
# - Avg Assembly time:  140.022447 s 
# - Avg Boundary time:  0.033517 s 
# - Avg Solution time:  687.072005 s 
# - Avg Total run time: 827.127969 s


# Prealloc (No-test):

# Times:<class 'pyoti.real.dmat'>
# - Avg Assembly time:  17.595636 s 
# - Avg Boundary time:  0.014864 s 
# - Avg Solution time:  28.491682 s 
# - Avg Total run time: 46.102182 s 

# Times:<class 'pyoti.static.mdnum2.mdmat2'>
# - Avg Assembly time:  29.271883 s 
# - Avg Boundary time:  0.058937 s 
# - Avg Solution time:  47.162110 s 
# - Avg Total run time: 76.492931 s 

# Times:<class 'pyoti.static.mdnum5.mdmat5'>
# - Avg Assembly time:  52.841292 s 
# - Avg Boundary time:  0.035432 s 
# - Avg Solution time:  102.099224 s 
# - Avg Total run time: 154.975949 s

# Times:<class 'pyoti.static.onumm1n5.omatm1n5'>
# - Avg Assembly time:  27.008188 s 
# - Avg Boundary time:  0.026359 s 
# - Avg Solution time:  54.162186 s 
# - Avg Total run time: 81.196733 s 


# Times:<class 'pyoti.static.onumm1n2.omatm1n2'>
# - Avg Assembly time:  27.926195 s 
# - Avg Boundary time:  0.026295 s 
# - Avg Solution time:  45.847667 s 
# - Avg Total run time: 73.800157 s 



# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  35.714642 s 
# - Avg Boundary time:  0.031543 s 
# - Avg Solution time:  143.801926 s 
# - Avg Total run time: 179.548111 s 

# Times:<class 'pyoti.sparse.matso'>
# - Avg Assembly time:  34.164061 s 
# - Avg Boundary time:  0.030414 s 
# - Avg Solution time:  36.347813 s 
# - Avg Total run time: 70.542288 s 

# Times:<class 'pyoti.sparse.matso'> (E = E+alg.e(1))
# - Avg Assembly time:  48.889351 s 
# - Avg Boundary time:  0.027572 s 
# - Avg Solution time:  46.364973 s 
# - Avg Total run time: 95.281895 s 

# Prealloc (TEST):
# Times:<class 'pyoti.static.onumm1n1.omatm1n1'>
# - Avg Assembly time:  27.116220 s 
# - Avg Boundary time:  0.025703 s 
# - Avg Solution time:  43.230549 s 
# - Avg Total run time: 70.372471 s 
# Times:<class 'pyoti.static.mdnum2.mdmat2'>
# - Avg Assembly time:  27.519817 s 
# - Avg Boundary time:  0.042431 s 
# - Avg Solution time:  45.346090 s 
# - Avg Total run time: 72.908338 s 
# Times:<class 'pyoti.static.onumm1n2.omatm1n2'>
# - Avg Assembly time:  28.312769 s 
# - Avg Boundary time:  0.025430 s 
# - Avg Solution time:  45.361990 s 
# - Avg Total run time: 73.700188 s 
# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  36.328263 s 
# - Avg Boundary time:  0.031260 s 
# - Avg Solution time:  110.400486 s 
# - Avg Total run time: 146.760009 s 

In [9]:
u = alg.zeros((u_res.shape[0]//2,3))
u[:,0:1] = u_res[ :u_res.shape[0]//2, : ]
u[:,1:2] = u_res[ u_res.shape[0]//2:, : ]

u[0,0:2] = np.nan
u.real

u_r = np.linalg.norm(u.real,axis=1).reshape((u.shape[0],-1))

In [10]:
u_r

array([[       nan],
       [0.00116128],
       [0.00116124],
       ...,
       [0.00111442],
       [0.00111483],
       [0.00111525]])

In [30]:
deriv = [2]
dudE = u.get_deriv(deriv)
dudE[0,0:2] = np.nan
dudE_r = np.linalg.norm(dudE.real,axis=1).reshape((dudE.shape[0],-1))
dudE

array([[            nan,             nan,  0.00000000e+00],
       [-8.14068660e-33,  7.14034146e-04,  0.00000000e+00],
       [ 7.14187525e-04, -3.42859698e-32,  0.00000000e+00],
       ...,
       [ 2.35887240e-03,  6.37834831e-06,  0.00000000e+00],
       [ 2.36292863e-03,  6.38933168e-06,  0.00000000e+00],
       [ 2.36698378e-03,  6.40021844e-06,  0.00000000e+00]])

In [12]:
ux_a, uy_a = analytic_solution(Th,E,nu,ri,Pi,ro,Po)
u_a = alg.zeros(u.shape)
u_a[:,0:1]=ux_a
u_a[:,1:2]=uy_a
u_a.real
u_ra = np.linalg.norm(u_a.real,axis=1).reshape((u_a.shape[0],-1))
u_ra

array([[       nan],
       [0.00116126],
       [0.00116126],
       ...,
       [0.00111442],
       [0.00111483],
       [0.00111525]])

In [31]:
dudE_a = u_a.get_deriv(deriv)
dudE_a[0,0:2] = np.nan
dudE_ra = np.linalg.norm(dudE_a.real,axis=1).reshape((dudE_a.shape[0],-1))
dudE_a

array([[           nan,            nan, 0.00000000e+00],
       [0.00000000e+00, 7.14095238e-04, 0.00000000e+00],
       [7.14095238e-04, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.35887319e-03, 6.37748431e-06, 0.00000000e+00],
       [2.36292960e-03, 6.38845128e-06, 0.00000000e+00],
       [2.36698485e-03, 6.39941512e-06, 0.00000000e+00]])

In [32]:
eps = 1e-120
u_rerr   =np.abs(u_r-u_ra)/u_ra
dudE_rerr=np.abs(dudE_r - dudE_ra)/(dudE_ra)


In [33]:
dudE_rerr

array([[           nan],
       [8.55522384e-05],
       [1.29236705e-04],
       ...,
       [3.34064348e-07],
       [4.12135939e-07],
       [4.50957877e-07]])

In [17]:
ux_a, uy_a = analytic_solution(Th,21e5,0.28,1,10,2,1000)
u_a = np.zeros_like(u)
u_a[:,0:1]=ux_a.real
u_a[:,1:2]=uy_a.real
u_a

array([[        nan,         nan,  0.        ],
       [-0.        , -0.00116126,  0.        ],
       [-0.00116126, -0.        ,  0.        ],
       ...,
       [-0.00104268, -0.00033854,  0.        ],
       [-0.00034957, -0.00104825,  0.        ],
       [-0.00104825, -0.00034957,  0.        ]])

In [26]:
import pyvista as pv
p = pv.BackgroundPlotter()
args_cbar = dict(height=0.75, vertical=True, position_x=0.05, 
                 position_y=0.05, interactive=False,
                 title_font_size=30, label_font_size=30)

In [94]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
# pv.set_plot_theme('default')
p.set_background("white")
grid = Th.to_pv(pd = [u.real],pd_names=['u'])
factor = 100
p.clear()

arrows = grid.glyph(scale="u", orient="u", factor = 100,tolerance=0.001)

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args=args_cbar)
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,color='grey', scalar_bar_args=args_cbar)
# p.add_mesh(arrows, lighting=False, stitle="u", scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()


In [19]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
# pv.set_plot_theme('default')
p.set_background("white")
grid = Th.to_pv(pd = [u.real],pd_names=['u'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [20]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u_a.real],pd_names=['u_analytic'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [34]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u_rerr],pd_names=['relative error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [52]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u.real-u_a.real],pd_names=['absolute error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [ ]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u.real-u_a.real],pd_names=['absolute error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [35]:
import pyvista as pv
p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE_rerr],pd_names=['relative error (deriv)'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [27]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE_rerr],pd_names=['Error dudE_oti'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [29]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE_a],pd_names=['dudE_analytic'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [36]:
import pyvista as pv
p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE-dudE_a],pd_names=['10th deriv  abs. error.'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [22]:
x = Th.x.real
y = Th.y.real

In [26]:
# Analytic solution
def analytic_solution(x,y,E,nu,ri,pi,ro,po):
    
    A = ri**2*ro**2*(po-pi)/(ro**2-ri**2)
    B = (ri**2*pi - ro**2*po)/(ro**2-ri**2)

    
    r =  (x**2+y**2)**0.5
    
    ur = (1+nu)*r/E*((1-2*nu)*B-A/(r**2))
    
    ux = ur*x/r
    uy = ur*y/r
    
    return ux, uy

In [29]:
import sympy as sym
Es, nus, ris, Pis, ros, Pos, xs, ys = sym.symbols("E nu r_i P_i r_o P_o x y")

In [30]:
uxs, uys = analytic_solution(xs,ys,Es,nus,ris,Pis,ros,Pos)

In [35]:
ux_eval = sym.lambdify([ xs, ys, Es, nus, ris, Pis, ros, Pos],uxs)
duxdE_eval = sym.lambdify([ xs, ys, Es, nus, ris, Pis, ros, Pos],uxs.diff(Es,1))

uy_eval = sym.lambdify([ xs, ys, Es, nus, ris, Pis, ros, Pos],uys)
duydE_eval = sym.lambdify([ xs, ys, Es, nus, ris, Pis, ros, Pos],uys.diff(Es,1))

In [36]:
ux_eval(1,0,E.real,nu.real,ri.real, Pi.real, ro.real, Po.real)

-0.001161264761904762

In [37]:
duxdE_eval(1,0,E.real,nu.real,ri.real, Pi.real, ro.real, Po.real)

5.529832199546485e-10